# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-278000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-278000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Bidding

In [2]:
vuln_ns, vuln_ew = False, False
hand = 'AQJ986.QJ952.7.8'

auction = ["PASS", "1N"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 2, 0, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '2C', 'insta_score': 0.751, 'expected_score': -47, 'adjustment': 38},
 {'call': '2S', 'insta_score': 0.11, 'expected_score': -28, 'adjustment': 6}]

In [3]:
### Samples consistent with the auction so far

for i in range(min(10,len(bid.samples))):
    print(bid.samples[i])


Tx.xx.AKxx.QJxxx Kxx.ATx.Qxxx.AK9 AQJ98x.QJ9xx.x.8 xx.K8x.JT98.Txxx 0.96538 
 P-1N-2C-P-2H-P-2S-P-P-P (9.6)  
 P-1N-2S-P-P-P (9.6) 
x.8xx.AJ8x.KTxxx Kxx.KTx.KQxx.AJx AQJ98x.QJ9xx.x.8 Txx.Ax.T9xx.Q9xx 0.99914 
 P-1N-2C-P-2H-P-2S-P-P-P (7.5)  
 P-1N-2S-P-P-P (7.5) 
Tx.Axx.QJ8x.T9xx Kxxx.KT8.Kx.AQxx AQJ98x.QJ9xx.x.8 x.xx.AT9xxx.KJxx 0.97966 
 P-1N-2C-2D-P-P-2S-3D-P-P-P (8.8)  
 P-1N-2S-3D-P-3N-P-P-P (7.6) 
xx.T8x.Axx.J9xxx Tx.AK.QT98x.AKxx AQJ98x.QJ9xx.x.8 Kxx.xxx.KJxx.QTx 0.53458 
 P-1N-2C-X-P-2D-2S-3D-P-P-P (10.0)  
 P-1N-2S-2N-P-3C-P-3N-P-P-P (8.8) 
Tx.Txx.AT8xx.xxx Kxx.A8x.Kxx.AK9x AQJ98x.QJ9xx.x.8 xx.Kx.QJ9x.QJTxx 0.99930 
 P-1N-2C-X-P-P-2S-3C-P-P-P (8.9)  
 P-1N-2S-3C-P-P-P (8.9) 
xx.A8.9xxx.KQTxx KTxx.Kx.KQJx.AJx AQJ98x.QJ9xx.x.8 x.Txxx.AT8x.9xxx 0.99779 
 P-1N-2C-P-2H-P-2S-P-P-P (9.0)  
 P-1N-2S-P-P-P (9.0) 
xxx.xx.KJ9x.9xxx Kx.K8xx.Axxx.AKx AQJ98x.QJ9xx.x.8 Tx.AT.QT8x.QJTxx 0.99620 
 P-1N-2C-X-P-2H-2S-2N-P-P-P (7.9)  
 P-1N-2S-3C-P-P-P (9.7) 
Tx.Tx.Jxx.QJ9xxx Kxx.Kxx.A8xx.AKx AQJ

West always has the same cards because it's your hand and it's known to you

In [4]:
# we got some samples above. if we want more, we can always get more
auction = ["PASS", "1N"]
sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(auction, 2)
print("Good quality:", good_quality)
for i in range(min(10,sample_hands.shape[0])):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])} {sorted_score[i]:.2f}')

Tx.xx.AKxx.QJxxx Kxx.ATx.Qxxx.AK9 AQJ98x.QJ9xx.x.8 xx.K8x.JT98.Txxx 0.97
x.8xx.AJ8x.KTxxx Kxx.KTx.KQxx.AJx AQJ98x.QJ9xx.x.8 Txx.Ax.T9xx.Q9xx 1.00
Tx.Axx.QJ8x.T9xx Kxxx.KT8.Kx.AQxx AQJ98x.QJ9xx.x.8 x.xx.AT9xxx.KJxx 0.98
xx.T8x.Axx.J9xxx Tx.AK.QT98x.AKxx AQJ98x.QJ9xx.x.8 Kxx.xxx.KJxx.QTx 0.53
Tx.Txx.AT8xx.xxx Kxx.A8x.Kxx.AK9x AQJ98x.QJ9xx.x.8 xx.Kx.QJ9x.QJTxx 1.00
xx.A8.9xxx.KQTxx KTxx.Kx.KQJx.AJx AQJ98x.QJ9xx.x.8 x.Txxx.AT8x.9xxx 1.00
xxx.xx.KJ9x.9xxx Kx.K8xx.Axxx.AKx AQJ98x.QJ9xx.x.8 Tx.AT.QT8x.QJTxx 1.00
Tx.Tx.Jxx.QJ9xxx Kxx.Kxx.A8xx.AKx AQJ98x.QJ9xx.x.8 xx.A8x.KQT9x.Txx 1.00
xx.xx.AJTxx.Axxx KT.AKT.Q9x.KJ9xx AQJ98x.QJ9xx.x.8 xxx.8xx.K8xx.QTx 1.00
Tx.xx.T8xx.Axxxx Kxx.A8x.AQJx.KT9 AQJ98x.QJ9xx.x.8 xx.KTx.K9xx.QJxx 1.00
